# Ansible

[Ansible](https://docs.ansible.com/) is a tool that allows you to automate some common operations on a set of hosts.

## Setup

Using ansible requires:

- Control node with ssh client and ansible installed.
- Managed node with ssh server, and python interpreter.
- Provide access from control node to managed node.

The role of the nodes in our examples will be to play docker containers. These containers require some preparation, so we'll create images.

For the managed node, we need to install and run the ssh server.

In [1]:
cat << EOF > ansible_files/managed_node_dockerfile

FROM python:3.10-alpine
RUN apk add -q openssh && ssh-keygen -A && mkdir /root/.ssh
CMD ["/usr/sbin/sshd", "-D"]

EOF

For control mode, we need to install the ssh client and ansible as a python package.

In [2]:
cat << EOF > ansible_files/control_node_dockerfile

FROM python:3.10-alpine
RUN apk add openssh-client && \\
    ssh-keygen -t rsa -N "" -f /root/.ssh/id_rsa && \\
    pip3 install ansible && \\
    # Add to the config statement that we don't need to enter if we want to save the new host.
    echo -e "Host *\n\tStrictHostKeyChecking no\n\n" > ~/.ssh/config

EOF

The following cell creates images that we'll use to consider ansible.

In [3]:
docker build -t managed_node \
    -f ./ansible_files/managed_node_dockerfile \
    ./ansible_files &> /dev/null
docker build -t control_node \
    -f ./ansible_files/control_node_dockerfile \
    ./ansible_files &> /dev/null

## Hello world

Here, without going too deep, let's look at the basic features of ansible.

To make ansible work we need:

- Control node with ssh client and ansible installed.
- Managed node with ssh server, and python interpreter.
- Provide access from control node to managed node.

The role of the nodes in our examples will be to play docker containers. The following cell creates docker containers and configures access between them.

In [4]:
docker network create ansible_test_network
docker run -itd --rm --name ansible_managed_node --network ansible_test_network managed_node
docker run -itd --rm --name ansible_control_node --network ansible_test_network control_node

export public_key=$(docker exec ansible_control_node cat /root/.ssh/id_rsa.pub)
docker exec ansible_managed_node sh -c "echo \"$public_key\" >> /root/.ssh/authorized_keys"

Error response from daemon: network with name ansible_test_network already exists
docker: Error response from daemon: Conflict. The container name "/ansible_managed_node" is already in use by container "81ccbf7d345c84f926b861d6729a5f716c3a2b359de0c9e1e83bcdc0ae1a0aa5". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.
docker: Error response from daemon: Conflict. The container name "/ansible_control_node" is already in use by container "7e532204ced6828a8b58f9928ba6267f9adf64ec2856f4c9d58540907787c38d". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


**Note** don't forget to stop containres after all.

In [81]:
docker stop ansible_managed_node ansible_control_node
docker network rm ansible_test_network

ansible_managed_node
ansible_control_node
ansible_test_network


### Inventory

Check [building an inventory](https://docs.ansible.com/ansible/latest/getting_started/get_started_inventory.html) page of documentation.

Inventories organise managed nodes into centralised files that provide Ansible with system information and network locations. An inventory file allows Ansible to manage a large number of hosts with a single command. So it's a file in which you list all the managed nodes.

Now in the control node we need to create an ansible config `inventory.ini` where we just specify managed node.

**Note** We have specified the python path with `ansible_python_interpreter=/usr/local/bin/python3.10`, it's not necessary but allows to avoid some warnings.

In [5]:
docker exec ansible_control_node sh -c  "
cat << EOF > inventory.ini
[myhosts]
ansible_managed_node ansible_python_interpreter=/usr/local/bin/python3.10
EOF"

Here `[myhosts]` is the name of the group of servers.

Finally, try pinging the managed nodes using the command `ansible <group of servers> -m ping -i inventory.ini`.

In [6]:
docker exec ansible_control_node ansible myhosts -m ping -i inventory.ini

ansible_managed_node | SUCCESS => {
    "changed": false,
    "ping": "pong"
}


As a result, we got the message `ansible_host | SUCCESS ...`, which tells us that we successfully ping the managed node.

### Playbook

Playbooks are files with instructions. So below is a playbook that creates a play called `My first play`. This play will ping all hosts and then print a message.

In [4]:
docker exec ansible_control_node sh -c  "
cat << EOF > playbook.yaml
- name: My first play
  hosts: myhosts
  tasks:
   - name: Ping my hosts
     ansible.builtin.ping:

   - name: Print message
     ansible.builtin.debug:
      msg: Hello world"

So now you can use the `ansible-playbook` command to play the playbook we have just created.

In [5]:
docker exec ansible_control_node ansible-playbook -i inventory.ini playbook.yaml


PLAY [My first play] ***********************************************************

TASK [Gathering Facts] *********************************************************
ok: [ansible_managed_node]

TASK [Ping my hosts] ***********************************************************
ok: [ansible_managed_node]

TASK [Print message] ***********************************************************
ok: [ansible_managed_node] => {
    "msg": "Hello world"
}

PLAY RECAP *********************************************************************
ansible_managed_node       : ok=3    changed=0    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



So there are tasks: `Ping my hosts` and `Print message`, just as we specified.

### Modules

Check the page [introduction to modules](https://docs.ansible.com/ansible/latest/module_plugin_guide/modules_intro.html).

Modules are discrete units of code that can be used in ansible playbooks or in the CLI as an argument to the `ansible` command.

As an example, let's consider the `command` module, which enables the execution of commands on a managed node. Here, we have the following details:

- The `-m` parameter specifies the module we want to execute.
- The `-a` parameter specifies the arguments for that module.

Sence of the command - creation a file called `secret_file`.

In [6]:
docker exec ansible_control_node \
    ansible myhosts \
        -i inventory.ini \
        -m command \
        -a "touch secret_file"

ansible_managed_node | CHANGED | rc=0 >>



Let's check if it appeared in the managed node.

In [7]:
docker exec ansible_managed_node ls /root

secret_file


## Copy files

You can copy files from master node to managed node by using corresponding ["ansible.builtin.copy" module](https://docs.ansible.com/ansible/latest/collections/ansible/builtin/copy_module.html).

---

The following cell creates a playbook that copies itself to the managed node:

In [4]:
docker exec ansible_control_node sh -c  "
cat << EOF > playbook.yaml
- name: Copy example
  hosts: myhosts
  tasks:
   - name: Copy task
     ansible.builtin.copy:
       src: ./playbook.yaml
       dest: /hello_secret_file"

Here is what such playbook run looks like:

In [5]:
docker exec ansible_control_node ansible-playbook -i inventory.ini playbook.yaml


PLAY [Copy example] ************************************************************

TASK [Gathering Facts] *********************************************************
ok: [ansible_managed_node]

TASK [Copy task] ***************************************************************
changed: [ansible_managed_node]

PLAY RECAP *********************************************************************
ansible_managed_node       : ok=2    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



As a result, you will find the copied file in the managed node:

In [6]:
docker exec ansible_managed_node cat hello_secret_file

- name: Copy example
  hosts: myhosts
  tasks:
   - name: Copy task
     ansible.builtin.copy:
       src: ./playbook.yaml
       dest: /hello_secret_file
